In [11]:
import pandas as pd
import numpy as np
import scanpy as sc

In [7]:
adata = sc.read_h5ad('scRNAseq_pancan_MALE_after_SCVI.h5ad')
adata

AnnData object with n_obs × n_vars = 497055 × 14090
    obs: 'SampleID', 'DataID', 'Age', 'Sex', 'Treatment', 'Organ', 'Cancertype', 'tissue_type', 'DataSampleID', 'celltype', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'batch', '_scvi_batch', '_scvi_labels', 'Celltype1', 'Celltype2', 'Celltype3', 'Cancertype_TCGA', 'RL_Prediction_Only9genes', 'LOY'
    var: 'mt-1', 'n_cells_by_counts-1', 'mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Cancertype_colors', 'Celltype1_colors', 'Celltype2_colors', 'DataID_colors', 'Organ_colors', 'Sex_colors', '_scvi_manager_uuid', '_scvi_uuid', 'celltype_colors', 'hvg', 'neighbors', 'rank_genes_groups', 'umap'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'
    obsp: 'connectivities', 'distances'

In [12]:
adata.obs['Celltype2_LOY'] = adata.obs['Celltype2'].astype('str')+'_'+adata.obs['LOY'].astype('str')

In [4]:
#DGE results
Epi = pd.read_csv('Extended_Data_Table_5_Epithlium_LOYvsWTY_DGE.csv',index_col=0)#Epithelium
data = pd.read_csv('ExtendedData_Table9_DGE.csv',index_col=0)#Other cells

In [5]:
#Combine results
Epi.columns = ['Epithelium_'+col for col in Epi.columns]
data = pd.concat([Epi,data],axis=1)
data

,Epithelium_LOY_SCR_names,Epithelium_LOY_SCR_logfoldchanges,Epithelium_LOY_SCR_pvals,Epithelium_LOY_SCR_pvals_adj,Epithelium_LOY_SCR_scores,Endothelium_LOY_SCR_names,Endothelium_LOY_SCR_logfoldchanges,Endothelium_LOY_SCR_pvals,Endothelium_LOY_SCR_pvals_adj,Endothelium_LOY_SCR_scores,...,MAIT_LOY_SCR_names,MAIT_LOY_SCR_logfoldchanges,MAIT_LOY_SCR_pvals,MAIT_LOY_SCR_pvals_adj,MAIT_LOY_SCR_scores,Monocyte_LOY_SCR_names,Monocyte_LOY_SCR_logfoldchanges,Monocyte_LOY_SCR_pvals,Monocyte_LOY_SCR_pvals_adj,Monocyte_LOY_SCR_scores
0,GAPDH,0.459859,0.0,0.0,70.546650,XIST,10.006480,0.000000e+00,0.000000e+00,64.165420,...,RPL41,1.123600,3.731617e-154,2.628924e-150,26.449152,MALAT1,0.461685,1.015523e-104,7.530902e-102,21.732315
1,ACTB,0.414018,0.0,0.0,65.135480,S100A2,3.722147,1.961521e-257,3.454728e-254,34.273304,...,RPL31,1.327024,5.575283e-119,2.618525e-115,23.191286,NEAT1,0.486352,5.161562e-104,3.463163e-101,21.657532
2,S100P,1.136207,0.0,0.0,61.890232,MALAT1,0.412729,1.501281e-236,1.923005e-233,32.841600,...,RPS4X,0.634212,7.770441e-116,2.737138e-112,22.877573,RPL41,0.389036,8.446235e-95,4.577210e-92,20.656994
3,KRT17,1.325077,0.0,0.0,59.592888,HSPB1,0.557924,6.478197e-205,4.804095e-202,30.545553,...,RPS29,0.804763,2.793711e-111,7.872678e-108,22.415266,NAMPT,0.573663,3.043359e-72,8.090742e-70,17.975277
4,FTL,0.314428,0.0,0.0,59.173890,KRT17,2.380481,4.405345e-204,3.103566e-201,30.482800,...,RPL36A,1.209311,3.539451e-90,7.124410e-87,20.136430,MT-ND2,0.289776,4.222861e-70,1.043862e-67,17.699602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14085,DDX3Y,-6.942289,0.0,0.0,-80.074300,CD99,-0.976461,1.271776e-298,2.986555e-295,-36.934920,...,MT-ND5,-1.482532,1.771278e-78,2.268846e-75,-18.754726,MT-ATP8,-1.691407,1.162461e-167,3.275814e-164,-27.598417
14086,RPL31,-0.717564,0.0,0.0,-80.082510,VIM,-0.765372,2.644985e-299,7.453568e-296,-36.977383,...,MT-CO2,-0.789815,6.221653e-83,8.766309e-80,-19.292400,DDX3Y,-12.728636,3.398499e-190,1.197121e-186,-29.416300
14087,RPS10,-0.925467,0.0,0.0,-83.973180,ALDOA,-1.745948,0.000000e+00,0.000000e+00,-42.461365,...,MT-CO3,-0.928753,8.322531e-87,1.302938e-83,-19.748140,ALDOA,-1.506888,1.354131e-190,6.359903e-187,-29.447529
14088,EIF1AY,-7.171783,0.0,0.0,-106.597570,DDX3Y,-10.271074,0.000000e+00,0.000000e+00,-46.772465,...,MT-ATP6,-1.322923,3.336710e-98,7.835707e-95,-21.032084,EEF1A1,-0.686648,2.009947e-209,1.416008e-205,-30.883171


In [22]:
#Mark non-zero expression
X = pd.DataFrame(adata.X.todense(),index=adata.obs_names,columns=adata.var_names)
X = X.gt(0).copy()
X['Celltype2_LOY'] = adata.obs['Celltype2_LOY'].copy()
#Get cell number with non-zero expression
expr_cnt = X.groupby('Celltype2_LOY').sum()
#Total cell number for each cell type
cnt = X.groupby('Celltype2_LOY').size()
#Get expression proportion
prop = expr_cnt.div(cnt,axis='rows')
prop.to_csv('scRNAseq_pancan_MALE_gene_expression_proportion_Endo2.csv')

In [14]:
def get_sig_genes(Expr_prop, res, CTs, lfc, padj, cutoff):
    Celltype_sig = {}
    for ct in CTs:
        #Select genes expressed significantly higher in LOY_SCR (WTY_SCR) groups
        genelist_LOY  = res.loc[((res.loc[:,ct+'_LOY_SCR_logfoldchanges']>lfc) & 
                                     (res.loc[:,ct+'_LOY_SCR_pvals_adj']<padj)),
                                ct+'_LOY_SCR_names'].to_list()
        genelist_WTY  = res.loc[((res.loc[:,ct+'_LOY_SCR_logfoldchanges']<-lfc) & 
                                     (res.loc[:,ct+'_LOY_SCR_pvals_adj']<padj)),
                                ct+'_LOY_SCR_names'].to_list()
        #Select genes expressed in less cutoff % of other cells
        sub_prop = pd.DataFrame(Expr_prop.loc[Expr_prop.index!=ct+'_LOY_SCR',:].max(),columns=['max_prop'])
        genelist_prop_LOY = sub_prop.index[sub_prop['max_prop']< cutoff/100]
        sub_prop = pd.DataFrame(Expr_prop.loc[Expr_prop.index!=ct+'_WTY_SCR',:].max(),columns=['max_prop'])
        genelist_prop_WTY = sub_prop.index[sub_prop['max_prop']< cutoff/100]
        #Get final list
        genelist = set(genelist_LOY).intersection(genelist_prop_LOY)
        Celltype_sig[ct+'_LOY_SCR'] = list(genelist)
        genelist = set(genelist_WTY).intersection(genelist_prop_WTY)
        Celltype_sig[ct+'_WTY_SCR'] = list(genelist)
        
    return Celltype_sig

In [91]:
CTs = ['CD4+_T_cell','CD8+_T_cell','Epithelium']
import math
Celltype_sig= get_sig_genes(prop,data, CTs, math.log(1.5,2), 0.05, 20)
    
#Keep genes with LFC>1 when having over 200 genes
genegrid_new = pd.DataFrame(index=adata.var_names)

for ctloy in Celltype_sig.keys():
    ct = ctloy[:len(ctloy)-len('_LOY_SCR')]
    loy = ctloy[-len('LOY_SCR'):]
    
    genelist = Celltype_sig[ctloy]
    genelist = data.loc[data[ct+'_LOY_SCR_names'].isin(genelist),:].copy()
    
    if(len(genelist)>200):
        if loy == 'LOY_SCR':
            genelist = genelist.loc[genelist[ct+'_LOY_SCR_logfoldchanges']>1,ct+'_LOY_SCR_names'].tolist()
        else:
            genelist = genelist.loc[genelist[ct+'_LOY_SCR_logfoldchanges']<-1,ct+'_LOY_SCR_names'].tolist()
    else:
        genelist = genelist.loc[:,ct+'_LOY_SCR_names'].tolist()

    genegrid_new[ctloy] = genegrid_new.index.isin(genelist)

genegrid_new.to_csv('LOY_Celltype_signature_genegrid.csv')